In [1]:
!pip install tensorflow

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys, os
from keras.layers import MaxPool2D
import matplotlib.pyplot as plt
import pandas
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# 数据集路径
train_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/val"


def load_images_from_directory(directory, target_size=(128, 128)):  # 增加输入尺寸
    images = []
    labels = []
    class_names = os.listdir(directory)  # 获取分类文件夹
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(directory, class_name)
        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            image = load_img(image_path, target_size=target_size)
            image = img_to_array(image) / 255.0  # 归一化到[0, 1]
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# 加载训练和验证数据
X_train, y_train = load_images_from_directory(train_dir, target_size=(128, 128))  # 增加输入尺寸
X_val, y_val = load_images_from_directory(val_dir, target_size=(128, 128))  # 增加输入尺寸




In [4]:
# 2. CNN模型建立（调整卷积核大小）
model = Sequential([
    Conv2D(input_shape=(64, 64, 3), filters=16, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dropout(0.2),
    Dense(240, activation='relu'),
    Dense(84, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# 3. 模型编译
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 回调机制
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.000001)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

# 自定义回调以计算和显示指标
class MetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_preds = np.argmax(self.model.predict(X_val), axis=1)
        val_labels = np.argmax(y_val, axis=1)
        precision = precision_score(val_labels, val_preds)
        recall = recall_score(val_labels, val_preds)
        f1 = f1_score(val_labels, val_preds)
        auc_roc = roc_auc_score(val_labels, val_preds)

        print(f"\nEpoch {epoch+1} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC-ROC: {auc_roc:.4f}")

metrics_callback = MetricsCallback()


NameError: name 'Callback' is not defined

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,  # 设置最大 epoch 数为 10
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[reduce_lr, early_stopping, metrics_callback],
    verbose=1
)

# 5. 绘制训练过程中的准确率和损失图
plt.figure()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.savefig('cat_dog_acc.png', dpi=600)

plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig('cat_dog_loss.png', dpi=600)
plt.show()